In [17]:
# notebook to test scraping with selenium on glassdoor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
# to find links
from bs4 import BeautifulSoup
import json
import requests
import urllib.request
import re
import time # to sleep
# to output a dataframe to file 
import pandas as pd
# fill this in with your job preferences!
# glassdoor search bar 
PREFERENCES = {
    "position_title": "Entwicklungsingenieur",
    "location": "Berlin, Deutschland"
}
# glassdoor login credentiels
from secret import glassdoor_credentials as s
LOGIN = {
    "username": s['email'],  # to be changed with the email
    "password": s['password'] # to be changed with the password
}
JOB_APP = {
    "first_name": "Sami",
    "last_name": "Dhiab",
    "fullname": "Sami Dhiab",
    "email": s['email'],
    "phone": "+4917666994604",
    "title": "M.Eng.Robotics",
    "company": "DLR",
    "resume": "C:/Users/Sami/Downloads/Resume-Sami-Dhiab.pdf", # change path to resume, for windows C:\\ should be present
    "linkedin": "https://www.linkedin.com/",
    "website": "www.youtube.com",
    "github": "https://github.com", 
    "twitter": "www.twitter.com",
    "city": "Berlin",
    "country": "Deutschland",
    "grad_month": '05',
    "grad_year": '2021',
    "university": "Beuth" 
}

In [18]:
# default selenium configuartions
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--disable-extensions")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--disable-blink-features")
options.add_argument("--disable-blink-features=AutomationControlled")
#options.add_argument("--headless") # if headless mode
options.add_argument("user-data-dir=selenium") # to store cookies between sessions
driver = webdriver.Chrome(options=options, executable_path='C:/Users/Sami/chromedriver.exe') # change path to chromedriver.exe


In [19]:
# browse the page and retrieve search resultst
driver.get('https://www.glassdoor.de/profile/login_input.htm?userOriginHook=HEADER_SIGNIN_LINK')
# fill out username and password 
try:    # if first time login
    driver.find_element_by_xpath("//*[@id='userEmail']").send_keys(LOGIN['username']) 
    driver.find_element_by_xpath("//*[@id='userPassword']").send_keys(LOGIN['password'])
# accept cookies and click login
    driver.find_element_by_xpath('//button[text()="Cookies akzeptieren"]').click()
    driver.find_element_by_xpath('//button[text()="Anmelden"]').click()
# keep waiting for user to log-in until the URL changes to user page
    while True:
        try:
            WebDriverWait(driver, 5).until(EC.url_contains("member"))
        except TimeoutException:
            break
except:
    print('you are already logged in')
    pass
# wait for the search bar to appear
try:
    # look for search bar fields and send elemnets
    #WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[@id='scBar']")))
    keyword=driver.find_element_by_xpath("//*[@id='sc.keyword']")
    keyword.send_keys(Keys.CONTROL + "a")
    keyword.send_keys(Keys.DELETE)
    keyword.send_keys(PREFERENCES['position_title']) # type in search bar
    location=driver.find_element_by_xpath("//*[@id='sc.location']")
    location.send_keys(Keys.CONTROL + "a")
    location.send_keys(Keys.DELETE)
    location.send_keys(PREFERENCES['location']) # type in location bar
    driver.find_element_by_xpath(" //*[@id='scBar']/div/button").click() # click search

except NoSuchElementException:
    print('check unfoudn elements')
# filter jobs by quick apply jobs
try:
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="DKFilters"]')))
    driver.find_element_by_xpath('//*[@id="DKFilters"]/div/div[1]/div[2]/div[1]/span[1]').click()
    driver.find_element_by_xpath('//*[@id="DKFilters"]/div/div[1]/div[2]/div[2]/div[10]/label').click()
    driver.find_element_by_xpath('//*[@id="DKFilters"]/div/div[1]/div[2]/div[1]/span[1]').click()
except ElementClickInterceptedException:
    pass

you are already logged in


In [20]:
# collect all links from current page using bs
time.sleep(1)
url = driver.current_url
hdr = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(url, headers=hdr)
infile = urllib.request.urlopen(req).read()
response = infile.decode('ISO-8859-1')
soup = BeautifulSoup(response)
# find all hrefs
allJobLinks = soup.findAll("a", {"class": "jobLink"})
allLinks = list(set([jobLink['href'] for jobLink in allJobLinks]))
#print(allLinks[0])
#print(type(allLinks[0]))
print("number of job found", len(allLinks))

number of job found 14


In [21]:
# iterarte all links to apply
try:
    for link in allLinks:
        url = "https://www.glassdoor.de"+link
        print("applying for the current jobpage: ", url)
        print()
        driver.get(url)
        # bypass popup if applying on main page
        try:
            driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span/svg').click()
        except NoSuchElementException:
            pass 
        # if redirecting to the jobpage 
        driver.find_element_by_xpath('//*[@id="JobView"]/div[1]/div[2]/div/div/div[2]/div/div[2]/div/div[1]/div[1]/span/button').click() # click schnellbewerbung
        try:
            driver.find_element_by_xpath('//*[@id="close-popup"]/text()').click() #close popup if already apllied for job
        except NoSuchElementException:
            pass
        driver.switch_to.frame(driver.find_element_by_xpath("//iframe[@title='Job application form container']")) #switch to integrated frame 1
        driver.switch_to.frame(driver.find_element_by_xpath("//iframe[@title='Job application form']")) #switch to second iframe
        # trying differents forms
        try:
            driver.find_element_by_xpath('//*[@id="input-applicant.name"]').send_keys(JOB_APP['fullname'])
            driver.find_element_by_xpath('//*[@id="input-applicant.email"]').send_keys(JOB_APP['email'])
            driver.find_element_by_xpath('//*[@id="input-applicant.phoneNumber"]').send_keys(JOB_APP['phone'])
            driver.find_element_by_xpath('//*[@id="input-applicant.location.city"]').send_keys(JOB_APP['city'])
            driver.find_element_by_xpath('//*[@id="input-applicant.jobTitle"]').send_keys(JOB_APP['title'])
            driver.find_element_by_xpath('//*[@id="input-applicant.companyName"]').send_keys(JOB_APP['company'])
        except NoSuchElementException:
            pass
        finally: 
            driver.find_element_by_name('applicant.fileUpload').send_keys(JOB_APP['resume'])

except ElementNotVisibleException:
    driver.close()
    pass

applying for the current jobpage:  https://www.glassdoor.de/partner/jobListing.htm?pos=103&ao=1136006&s=58&guid=000001785cdd3d6b8f15b95d9ed2621c&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_66eb5fd1&cb=1616465706730&jobListingId=4037453203

applying for the current jobpage:  https://www.glassdoor.de/partner/jobListing.htm?pos=107&ao=1136006&s=58&guid=000001785cdd3d6b8f15b95d9ed2621c&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_a3acb6a2&cb=1616465706731&jobListingId=4006115705

applying for the current jobpage:  https://www.glassdoor.de/partner/jobListing.htm?pos=114&ao=1136006&s=58&guid=000001785cdd3d6b8f15b95d9ed2621c&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_37a88eeb&cb=1616465706733&jobListingId=4015654512

applying for the current jobpage:  https://www.glassdoor.de/partner/jobListing.htm?pos=111&ao=1136006&s=58&guid=000001785cdd3d6b8f15b95d9ed2621c&src=GD_JOB_AD&t=SR&vt=w&ea=1&cs=1_aa5d48bf&cb=1616465706732&jobListingId=3797106886

applying for the current jobpage:  https://www.glassdoor.de/partner/jobListing.h

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="applicant.fileUpload"]"}
  (Session info: chrome=89.0.4389.90)
